## Decoding from different number systems

Saying that we want to represent a number in base $n$, say base $2$, means that we wish to represent numbers using only $n$ symbols. For example, if $n=2$, we wish to represent any number but only using two different symbols, like $\{0,1\}$, if $n=10$, we wish to represent any number using the symbols in $\{0,1,2,3,4,5,6,7,8,9\}$. If say $n=16$, we wish to represent any number using 16 different symbols like $\{0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f\}$.

The number system we humans use is in base 10, which is convenient for arbitrary reasons like the fact we generally have 10 fingers. However, for a computer to save numbers it is more convenient to use only two symbols rather than 10. Why? Because saving two symbols physically can be done by having current (1) or not having it (0) in a transistor, or having (1) or not having (0) voltage, which is a very easy thing yo verify even with a tolerance for error. If we instead wanted to encode 10 different symbols with the state of a transistor, we would require the reader of the transistor to be able to distinguish with precision 10 different current flow ranges in the transistor. Which, as you may know, in a transistor of 10 nm is not an easy task. Therefore, the computer internally works with numbers in binary, aka, $n=2$.

Let us denote by $(\cdot)_n$ the number $\cdot$ in $n$ base representation. If we do not specify the base with these parenthesis, we will assume it is in decimal base. This means that for example:
$$
(509.389)_{10}=5\cdot 10^2+0\cdot 10^1 + 9\cdot 10^0+3\cdot 10^-1+8\cdot 10^{-2}+9\cdot 10^{-3}=589.389
$$

or

$$
(110.01)_{2}=1\cdot 2^2+1\cdot 2^1 + 0\cdot 2^0+0\cdot 10^-1+1\cdot 10^{-2}=26.05859375
$$

or

$$
(1a.0f)_{16}=1\cdot 16^1+10\cdot 16^0+ 0\cdot 16^{-1}+15\cdot 16^{-2}=26.05859375
$$

As you can notice with this last example, if we use a base with more symbols than 10, it is important to know the order in which they are settled to make the correspondance to base 10. That is, we need to know $(a)_{16}=10$ or $(f)_{16}=15$. So when we design a number system, we must fix the order of the symbols we will use, such as fixing that $\{0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f\}$ is the **ordered** set of symbols in base 16, where the index of the element gives us the integer "in decimal base".

Definition of the convert to decimal function:

In [1]:
def convert_to_decimal(integer_symbols, fractional_symbols, base):
    decimal_values = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
    'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18,
    'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27,
    's': 28, 't': 29, 'u': 30
}

    def convert_symbol(symbol):
        if symbol in decimal_values:
            return decimal_values[symbol]
        else:
            raise ValueError("Invalid symbol: {}".format(symbol))

    integer_decimal = 0
    fractional_decimal = 0

    power = len(integer_symbols) - 1

    # Convert integer part
    for symbol in integer_symbols:
        integer_decimal += convert_symbol(symbol) * (base ** power)
        power -= 1

    power = -1

    # Convert fractional part
    for symbol in fractional_symbols:
        fractional_decimal += convert_symbol(symbol) * (base ** power)
        power -= 1

    decimal_number = integer_decimal + fractional_decimal
    return decimal_number

Asks basis and values:

In [3]:
print("***Decimal to base n (n>31) decoding calculator***")
print("Enter the ordered set of symbols")
num = input()

if '.' in num:
    parte_entera, parte_decimal = num.split('.')
    lista_decimal = [digito for digito in parte_decimal]
else:
    parte_entera = num
    lista_decimal = []

lista_entera = [digito for digito in parte_entera]

print("Enter the desired base")
base = int(input())

decimal_number = convert_to_decimal(lista_entera, lista_decimal, base)

print("Decimal number:", decimal_number)

***Decimal to base n (n>31) decoding calculator***
Enter the ordered set of symbols
1a.0f
Enter the desired base
16
Decimal number: 26.05859375


## Encoding to Alien number systems

Doing the opposite is not that trivial, but here is the **algorithm** explained with an example:

**(a)** For the **integer part** of the input decimal float: Say we wish to compute 3024 in base 23.
- We have that the whole division gives 3024 // 23 = 131 with residual 3024 % 23 = 11
- Get the residual of the division and save it to a list.
- Now take the whole division part and repeat the step: 131 // 23 = 5 with residual 131 % 23 = 16.
- Repeat these until you arrive to a whole division equal to 0. In our case we would need a step more: we would have saved 16 in the list of residuals and then 5 // 23 = 0 and 5 % 23 = 5. We append to the list also this last residual 5.
- Take the list of residuals [11,16,5]. From the first to the last, they will be the digit that should be multiplied by the base to the power of the index. Aka, $3024=11\cdot 23^0+16\cdot 23^1 + 5\cdot 23^2$. This means that we have it, since the residuals will always be smaller than the base. We just need to get the symbol corresponding to each number. In this case, if in base 23 we used the symbols '0123456789abcdefghijklm', we would have 11 is b, 16 is g and 5 is 5. Leaving $(3024)_{10}=(5gb)_{23}$.

**(b)** For the **decimal part** of the input decimal float: Say we wish to compute 2.7182 in base 23.
- The whole part is obtained with (a), in this case we have 2 in base 23 is 2.
- Drop the whole part, that is, take 0.7182 and compute its product with the base: $0.7182\cdot 23= 16.5186$
- Take the whole part 16 to a list and repeat again for the fractional part: $0.5186\cdot 23=11.9278$
- Take the whole part 11 to a list and repeat again for the fractional part: $0.9278\cdot 23 = 21.33939$
- And repeat and repeat until rather you get 0.0 in the decimal part or you arrive to the maximum decimal places you wish to represent. Note that a number that has a finite number of fractional digits in a base may have an infinite number in another base. This is possibly the case for 2.7182.
- The numbers you gathered in the list [16,11,21,..] are the decimal places for the new base representation, since: $0.7182=16\cdot 23^{-1}+11\cdot 23^{-2}+21\cdot 23^{-3}+...$. 
- Just get the symbol corresponding to each number using it as an index in the symbol list, for example. We would have that 2 is 2, 16 is g, 11 is b and 21 is l so $(2.7182)_{10}=(2.gbl\cdots)_{26}$.

Definition of the convert to decimal function:

In [4]:
def convert_to_decimal(integer_symbols, fractional_symbols, base):
    decimal_values = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
    'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18,
    'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27,
    's': 28, 't': 29, 'u': 30
}

    def convert_symbol(symbol):
        if symbol in decimal_values:
            return decimal_values[symbol]
        else:
            raise ValueError("Invalid symbol: {}".format(symbol))

    integer_decimal = 0
    fractional_decimal = 0

    
    # Convert integer part
    div = []
    symbol=int(parte_entera)
    while symbol // base > 0:
        division = symbol % base
        div.append(division)
        symbol = symbol // base
    div.append(symbol)

    converted_symbols = []
    for num in div:
        for key, value in decimal_values.items():
            if value == num:
                converted_symbols.append(key)
                break

    converted_symbols.reverse()

    integer_decimal =''.join(converted_symbols)

    # Convert fractional part
    count=0
    mult=[]
    fraction_symbols=float(parte_decimal)
    while count < 6 and fraction_symbols % 1 != 0:
        fraction_symbols = fraction_symbols * base
        mult.append(int(fraction_symbols))
        fraction_symbols = fraction_symbols- int(fraction_symbols)
        count += 1
    
    converted_symbols2 = []
    for num in mult:
        for key, value in decimal_values.items():
            if value == num:
                converted_symbols2.append(key)
                break

    fractional_decimal = ''.join(converted_symbols2)
    
    decimal_number = str(integer_decimal) +"."+str(fractional_decimal)

    return decimal_number

Asks basis and values:

In [8]:
print("***Decimal to base n (n<31) encoding calculator***")
print("Enter the nuber in decimal base")
num = input()

if '.' in num:
    parte_entera, parte_decimal = num.split('.')
    parte_decimal= float("0." + str(parte_decimal))
else:
    parte_entera = num 
    parte_decimal = 0

     
print("Enter the base")
base = int(input())

decimal_number = convert_to_decimal(parte_entera, parte_decimal, base)

print("Number in base", base,":", decimal_number)
    

***Decimal to base n (n<31) encoding calculator***
Enter the nuber in decimal base
26.05859375
Enter the base
16
Number in base 16 : 1a.0f
